# LongContextReorder

내용이 들어갈 때
20개의 retriever를 했을때, 중요도를 기준으로 내림차순으로 배치를 하게되면, 앞에와 뒤에 배치 되었을때 많이 참고한다, 가운데 들어가는 내용은 적게 참고한다.

즉 relevant information을 앞에부분을 잘 참고하고, 중간부분보다, 끝부분에서 더욱 잘 참고한다는 뜻이다.

그렇다면 관련성이 제일 떨어지는 문서를 더 참고를 많이하게 된다. 따라서 순위를 재조정 해야한다. 관련성이 떨어지면 가운데 배치하는 식으로!!

https://arxiv.org/pdf/2307.03172

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_community.document_transformers import LongContextReorder
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

texts = [
    "이건 그냥 내가 아무렇게나 적어본 글입니다.",
    "사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.",
    "아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.",
    "챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.",
    "챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.",
    "애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.",
    "ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.",
    "비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다.",
    "ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.",
    "FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.",
]

retriever = Chroma.from_texts(texts, embedding=embeddings).as_retriever(
    search_kwargs={"k":10}
)

In [3]:
query = "ChatGPT에 대해서 무엇을 말할 수 있나요?"

docs = retriever.invoke(query)
docs

[Document(page_content='ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.'),
 Document(page_content='ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.'),
 Document(page_content='사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.'),
 Document(page_content='챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.'),
 Document(page_content='이건 그냥 내가 아무렇게나 적어본 글입니다.'),
 Document(page_content='챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.'),
 Document(page_content='비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다.'),
 Document(page_content='아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.'),
 Document(page_content='애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.'),
 Document(page_content='FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.')]

`LongContextReorder` 클래스의 인스턴스인 reordering을 생성합니다.

- `reordering.transform_documents(docs)`를 호출하여 문서 목록 docs를 재정렬합니다.

- 덜 관련된 문서는 목록의 중간에 위치하고, 더 관련된 문서는 시작과 끝에 위치하도록 재정렬됩니다.

In [4]:
reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(docs)

reordered_docs

[Document(page_content='ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.'),
 Document(page_content='챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.'),
 Document(page_content='챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.'),
 Document(page_content='아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.'),
 Document(page_content='FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.'),
 Document(page_content='애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.'),
 Document(page_content='비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다.'),
 Document(page_content='이건 그냥 내가 아무렇게나 적어본 글입니다.'),
 Document(page_content='사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.'),
 Document(page_content='ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.')]

# context reordering을 사용하여 질의-응답 체인 생성

In [5]:
# 인덱스와 원소를 차례대로 반복자 객체를 반환해주는 함수.
def format_docs(docs):
    return "\n".join([doc.page_content for i, doc in enumerate(docs)])

In [6]:
print(format_docs(docs))

ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.
ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.
사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.
챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.
이건 그냥 내가 아무렇게나 적어본 글입니다.
챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.
비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다.
아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.
애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.
FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.


In [7]:
# 이런식으로 변경해서 넣을것이다.
# enumerate: 반복문에서 인덱스와 원소를 포함하여 return해주는 함수. 인덱스와 값을 동시에 접근하면서 루프를 돌릴떄 사용한다.
# join: 문자열 리스트를 하나의 문자열로 결합한다.
def format_docs(docs):
    return "\n".join(
        [
            f"[{i}] {doc.page_content} [source: skyop45@naver.com]"
            for i, doc in enumerate(docs)
        ]
    )
#  [source: skyop45@naver.com] 은 metadata

def reorder_documents(docs):
    # 재정렬
    reordering = LongContextReorder()
    reordered_docs = reordering.transform_documents(docs)
    combined = format_docs(reordered_docs)
    print(combined)
    return combined

In [15]:
def format_docs(docs):
    return "\n".join(
        [
            f"[{i}] {doc.page_content}"
            for i, doc in enumerate(docs)
        ]
    )

def reorder_documents(docs):
    reordering = LongContextReorder()
    reordered_docs = reordering.transform_documents(docs)
    combined = format_docs(reordered_docs)
    print(combined)
    return combined

In [9]:
_ = reorder_documents(docs)

[0] ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다. [source: skyop45@naver.com]
[1] 챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다. [source: skyop45@naver.com]
[2] 챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다. [source: skyop45@naver.com]
[3] 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다. [source: skyop45@naver.com]
[4] FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다. [source: skyop45@naver.com]
[5] 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다. [source: skyop45@naver.com]
[6] 비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다. [source: skyop45@naver.com]
[7] 이건 그냥 내가 아무렇게나 적어본 글입니다. [source: skyop45@naver.com]
[8] 사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다. [source: skyop45@naver.com]
[9] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다. [source: skyop45@naver.com]


In [10]:
from langchain.prompts import ChatPromptTemplate
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

template = """
Given this text extracts:
{context}

_____
Please answer the following question:
{question}
Answer in the Korean.
"""

prompt = ChatPromptTemplate.from_template(template)

# 질문을 던지고 검색을 하면, reorder_documents를 통해서 순위 재조정을 한다.
chain = (
    {
        "context": RunnablePassthrough()
        | retriever
        | RunnableLambda(reorder_documents),
        "question": RunnablePassthrough(),
        #"language": itemgetter("language")
    }
    | prompt
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

In [11]:
answer2 = chain.invoke("ChatGPT에 대해 무엇을 말해줄 수 있나요?")

[0] ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다. [source: skyop45@naver.com]
[1] 챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다. [source: skyop45@naver.com]
[2] 챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다. [source: skyop45@naver.com]
[3] 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다. [source: skyop45@naver.com]
[4] FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다. [source: skyop45@naver.com]
[5] 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다. [source: skyop45@naver.com]
[6] 비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다. [source: skyop45@naver.com]
[7] 이건 그냥 내가 아무렇게나 적어본 글입니다. [source: skyop45@naver.com]
[8] 사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다. [source: skyop45@naver.com]
[9] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다. [source: skyop45@naver.com]


In [12]:
from langchain.prompts import ChatPromptTemplate
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

# 프롬프트 템플릿
template = """Given this text extracts:
{context}

-----
Please answer the following question:
{question}

Answer in the Korean.
"""

# 프롬프트 정의
prompt = ChatPromptTemplate.from_template(template)

# Chain 정의
chain = (
    {
        "context": itemgetter("question")
        | retriever
        | RunnableLambda(reorder_documents),  # 질문을 기반으로 문맥을 검색합니다.
        "question": itemgetter("question"),  # 질문을 추출합니다.
        #"language": itemgetter("language"),  # 답변 언어를 추출합니다.
    }
    | prompt  # 프롬프트 템플릿에 값을 전달합니다.
    | ChatOpenAI(model="gpt-4o-mini")  # 언어 모델에 프롬프트를 전달합니다.
    | StrOutputParser()  # 모델의 출력을 문자열로 파싱합니다.
)

In [13]:
answer = chain.invoke({"question": "ChatGPT에 대해 무엇을 말해줄 수 있나요?"})

[0] ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다. [source: skyop45@naver.com]
[1] 챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다. [source: skyop45@naver.com]
[2] 챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다. [source: skyop45@naver.com]
[3] 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다. [source: skyop45@naver.com]
[4] FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다. [source: skyop45@naver.com]
[5] 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다. [source: skyop45@naver.com]
[6] 비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다. [source: skyop45@naver.com]
[7] 이건 그냥 내가 아무렇게나 적어본 글입니다. [source: skyop45@naver.com]
[8] 사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다. [source: skyop45@naver.com]
[9] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다. [source: skyop45@naver.com]
